# Agent V3: The Skilled

A monolithic agent with progressive disclosure via skills. Instead of loading all
specialized instructions into the system prompt upfront, the agent discovers skills
at startup (metadata only) and activates them on demand (loading full SKILL.md body).
Extends the Planner with activate_skill: file tools (9) + sandbox tools (1) + todo tools (6) + skill tool (1) = 17 tools.

In [ ]:
from pathlib import Path

from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.config.config import PROMPTS_DIR
from agentic_patterns.core.prompt import load_prompt
from agentic_patterns.core.skills.registry import SkillRegistry
from agentic_patterns.core.skills.tools import get_skill_instructions, list_available_skills
from agentic_patterns.tools.file import get_all_tools as get_file_tools
from agentic_patterns.tools.sandbox import get_all_tools as get_sandbox_tools
from agentic_patterns.tools.todo import get_all_tools as get_todo_tools

## Skill Discovery

Discovery loads only metadata (name + description). Full instructions stay on disk until the agent activates a skill.

In [ ]:
skills_root = Path("../skills/skills-demo")
registry = SkillRegistry()
registry.discover([skills_root])

print("Discovered skills:")
for meta in registry.list_all():
    print(f"  {meta}")

## Agent Definition

The system prompt includes the skill catalog (one-liner per skill). The `activate_skill`
tool is defined inline as a closure over the registry -- when the agent calls it, the full
SKILL.md body enters the context.

In [ ]:
skill_catalog = list_available_skills(registry)


def activate_skill(skill_name: str) -> str:
    """Activate a skill by name to load its full instructions into context."""
    instructions = get_skill_instructions(registry, skill_name)
    if instructions is None:
        return f"Skill '{skill_name}' not found. Use the skill catalog to see available skills."
    return instructions


system_prompt = load_prompt(
    PROMPTS_DIR / "the_complete_agent" / "agent_skilled.md",
    skill_catalog=skill_catalog,
)

tools = get_file_tools() + get_sandbox_tools() + get_todo_tools() + [activate_skill]
print(f"Total tools: {len(tools)}")
for t in tools:
    print(f"  {t.__name__}")

In [ ]:
import shutil
from agentic_patterns.core.config.config import WORKSPACE_DIR
from agentic_patterns.core.user_session import get_user_id, get_session_id

workspace_dir = WORKSPACE_DIR / get_user_id() / get_session_id()
if workspace_dir.exists():
    shutil.rmtree(workspace_dir)

agent = get_agent(system_prompt=system_prompt, tools=tools)

## Task: Code Review via Skill Activation

Ask the agent to write a script and then review it for security issues.
The agent should activate the code-review skill before attempting the review.

In [ ]:
prompt = """Write a Python script that reads user input and evaluates it as a math expression.
Save it to /workspace/calc.py. Then activate the code-review skill and review the script
for security issues. Summarize the findings."""

agent_run, nodes = await run_agent(agent, prompt, verbose=True)

print("\n--- Agent Output ---")
print(agent_run.result.output)

## Verify Workspace

In [ ]:
from agentic_patterns.core.workspace import list_workspace_files, read_from_workspace

for path in sorted(list_workspace_files("*")):
    print(f"--- {path} ---")
    print(read_from_workspace(path))
    print()